In [10]:
# loss函数的表现形式：

# 目标得到good-fit 模型

In [11]:
# 环境配置
# conda create -n tf_env python=3.10
# conda activate tf_env
# pip install tensorflow==2.13
# pip install "pyOpenSSL<25"


# conda install ipykernel  #if run in cell



In [ ]:
# -*- coding: utf-8 -*-
from tensorflow.keras.models import Model  # type: ignore
from tensorflow.keras.preprocessing import sequence # type: ignore
from tensorflow.keras.layers import Input # type: ignore
from tensorflow.keras.layers import Dense, Activation# type: ignore
from tensorflow.keras.layers import Embedding # type: ignore
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D # type: ignore
from tensorflow.keras.datasets import imdb # type: ignore
from tensorflow.keras import optimizers # type: ignore
import matplotlib # type: ignore
matplotlib.use('agg')
import matplotlib.pyplot as plt # type: ignore

In [13]:
# set parameters:
batch_size = 32
embedding_dims = 200
filters = 250
kernel_size = 3

In [14]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=top_words)
# pad dataset to a maximum review length in words
max_words = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)


In [15]:
# Build model
sentence = Input(batch_shape=(None, max_words), dtype='int32', name='sentence')
embedding_layer = Embedding(top_words, embedding_dims, input_length=max_words)
sent_embed = embedding_layer(sentence)
conv_layer = Conv1D(filters, kernel_size, padding='valid', activation='relu')
sent_conv = conv_layer(sent_embed)
sent_pooling = GlobalMaxPooling1D()(sent_conv)
sent_repre = Dense(250)(sent_pooling)
sent_repre = Activation('relu')(sent_repre)
sent_repre = Dense(1)(sent_repre)
pred = Activation('sigmoid')(sent_repre)
model = Model(inputs=sentence, outputs=pred)

In [16]:
#underfit
epochs = 30
sgd = optimizers.SGD(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [17]:
# fit the model
history = model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, verbose=1, validation_data=(x_test, y_test))
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/30
782/782 [==============================] - 62s 79ms/step - loss: 0.6927 - accuracy: 0.5138 - val_loss: 0.6920 - val_accuracy: 0.5794
Epoch 2/30
782/782 [==============================] - 64s 82ms/step - loss: 0.6915 - accuracy: 0.5399 - val_loss: 0.6907 - val_accuracy: 0.6070
Epoch 3/30
782/782 [==============================] - 61s 78ms/step - loss: 0.6898 - accuracy: 0.5751 - val_loss: 0.6888 - val_accuracy: 0.6192
Epoch 4/30
782/782 [==============================] - 58s 75ms/step - loss: 0.6873 - accuracy: 0.6198 - val_loss: 0.6855 - val_accuracy: 0.6845
Epoch 5/30
782/782 [==============================] - 58s 74ms/step - loss: 0.6824 - accuracy: 0.6605 - val_loss: 0.6785 - val_accuracy: 0.6820
Epoch 6/30
782/782 [==============================] - 58s 75ms/step - loss: 0.6705 - accuracy: 0.6627 - val_loss: 0.6600 - val_accuracy: 0.6264
Epoch 7/30
782/782 [==============================] - 58s 74ms/step - loss: 0.6478 - accuracy: 0.6289 - val_loss: 0.6373 - val_accuracy:

In [18]:
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.savefig('./loss.png')
# plt.show ()
